<a href="https://colab.research.google.com/github/bcury/Machine-Learning-Projects/blob/main/Pipelines_CrossValidation_RedesNeurais_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 25 kB/s 
     |████████████████████████████████| 198 kB 44.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=75f41002c9345dc6a89b40b8f46ba265ffdbe7ac22b00f034b3df53e6488d20d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [38]:
import pyspark,  findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('pipe').getOrCreate()

In [39]:
iris = spark.read.csv('iris.csv', header=True, inferSchema=True, sep=',')
iris.show()

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|
|        5.4|       3.7|        1.5|       0.2|Iris-setosa|
|        4.8|       3.4|        1.6|       0.2|Iris-setosa|
|        4.8|       3.0|        1.4|       0.1|Iris-setosa|
|        4.3|       3.0|        1.1|    

In [40]:
iris_treino, iris_teste = iris.randomSplit([0.7,0.3])

In [41]:
from pyspark.ml.feature import VectorAssembler

In [42]:
vector = VectorAssembler(inputCols=['sepallength','sepalwidth','petallength','petalwidth'],outputCol='independente')

In [43]:
from pyspark.ml.feature import StringIndexer

In [45]:
indexer = StringIndexer(inputCol='class',outputCol='dependente')

In [46]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [62]:
mlp = MultilayerPerceptronClassifier(maxIter=1000, layers=[4,5,4,3], featuresCol='independente',labelCol='dependente')

In [63]:
from pyspark.ml import Pipeline

In [64]:
pipe = Pipeline(stages=[vector, indexer, mlp])

In [65]:
modelo = pipe.fit(iris_treino)

In [66]:
previsao = modelo.transform(iris_teste)


In [67]:
previsao.select('independente','dependente','rawPrediction','probability','prediction').show()

+-----------------+----------+--------------------+--------------------+----------+
|     independente|dependente|       rawPrediction|         probability|prediction|
+-----------------+----------+--------------------+--------------------+----------+
|[4.4,3.0,1.3,0.2]|       2.0|[-170.58126558196...|[1.00283734656068...|       2.0|
|[4.4,3.2,1.3,0.2]|       2.0|[-170.58126558196...|[1.00283734656068...|       2.0|
|[4.8,3.4,1.6,0.2]|       2.0|[-170.58126558196...|[1.00283734656068...|       2.0|
|[4.8,3.4,1.9,0.2]|       2.0|[-170.58126558196...|[1.00283734656068...|       2.0|
|[4.9,3.1,1.5,0.1]|       2.0|[-170.58126558196...|[1.00283734656068...|       2.0|
|[4.9,3.1,1.5,0.1]|       2.0|[-170.58126558196...|[1.00283734656068...|       2.0|
|[4.9,3.1,1.5,0.1]|       2.0|[-170.58126558196...|[1.00283734656068...|       2.0|
|[5.0,3.3,1.4,0.2]|       2.0|[-170.58126558196...|[1.00283734656068...|       2.0|
|[5.1,2.5,3.0,1.1]|       1.0|[-170.58126558196...|[1.00283734656068...|    

In [68]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [69]:
performance = MulticlassClassificationEvaluator(labelCol='dependente',predictionCol='prediction',metricName='accuracy')
acuracia = performance.evaluate(previsao)
print(acuracia)

0.9302325581395349


interesse esse problema com pipeline

agora vamos fazer TUNNING de cross validation: testar parametros pra ver como fica o modelo

In [70]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [75]:
grid = ParamGridBuilder().addGrid(mlp.maxIter,[10,100,1000]).addGrid(mlp.layers,[[4,5,4,3],[4,4,3]]).build()

In [76]:
cross_validation = CrossValidator(estimator=pipe,estimatorParamMaps=grid,evaluator=performance,numFolds=3)

In [77]:
model = cross_validation.fit(iris_treino)

In [78]:
prev = model.transform(iris_teste)
prev.select('rawPrediction','probability','prediction').show()

+--------------------+--------------------+----------+
|       rawPrediction|         probability|prediction|
+--------------------+--------------------+----------+
|[-99.878099298349...|[1.49875900793774...|       2.0|
|[-99.878225954565...|[1.49879872162741...|       2.0|
|[-99.878470315454...|[1.49887534517942...|       2.0|
|[-99.878482128457...|[1.49887904944854...|       2.0|
|[-99.878405500756...|[1.49885502103633...|       2.0|
|[-99.878405500756...|[1.49885502103633...|       2.0|
|[-99.878405500756...|[1.49885502103633...|       2.0|
|[-99.878484669955...|[1.49887984640157...|       2.0|
|[-13.616946742329...|[2.73421684403939...|       1.0|
|[-99.878530844878...|[1.49889432582500...|       2.0|
|[-99.878544864974...|[1.49889872224104...|       2.0|
|[-99.878591826139...|[1.49891344839922...|       2.0|
|[-99.878594859346...|[1.49891439956205...|       2.0|
|[-99.878545357590...|[1.49889887671575...|       2.0|
|[-99.878574316437...|[1.49890795766165...|       2.0|
|[-99.8785

In [79]:
new_performance = MulticlassClassificationEvaluator(labelCol='dependente',predictionCol='prediction',metricName='accuracy')

In [80]:
acuracia = new_performance.evaluate(prev)

In [81]:
print(acuracia)

0.9534883720930233


a validação cruzada melhorou nosso modelo com TUNING de hiperparametro